In [ ]:
!pip install accelerate -U

In [ ]:
!pip install -q datasets pynvml

In [2]:
import numpy as np
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, logging
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo
import torch

# Initialize GPU monitoring
nvmlInit()

def print_gpu_utilization():
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

# Create dummy data
seq_len, dataset_size = 512, 512
dummy_data = {
    "input_ids": np.random.randint(100, 30000, (dataset_size, seq_len)),
    "labels": np.random.randint(0, 1, (dataset_size)),
}
ds = Dataset.from_dict(dummy_data)
ds.set_format("pt")

# Load model
model = AutoModelForSequenceClassification.from_pretrained("bert-large-uncased").to("cuda")

# Training arguments without gradient checkpointing
default_args = {
    "output_dir": "tmp",
    "evaluation_strategy": "steps",
    "num_train_epochs": 1,
    "log_level": "error",
    "report_to": "none",
}
training_args = TrainingArguments(per_device_train_batch_size=4, **default_args)
trainer = Trainer(model=model, args=training_args, train_dataset=ds)

# Train and summarize
result = trainer.train()
print_summary(result)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `

Step,Training Loss,Validation Loss


Time: 177.67
Samples/second: 2.88
GPU memory occupied: 7982 MB.


In [3]:
# Reset the environment (necessary to ensure accurate memory usage reporting)
torch.cuda.empty_cache()

# Training arguments with gradient checkpointing
training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    **default_args
)
trainer = Trainer(model=model, args=training_args, train_dataset=ds)

# Train and summarize
result = trainer.train()
print_summary(result)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'train_runtime': 266.2248, 'train_samples_per_second': 1.923, 'train_steps_per_second': 0.481, 'train_loss': 0.04136791452765465, 'epoch': 1.0}
Time: 266.22
Samples/second: 1.92
GPU memory occupied: 6834 MB.


# Exercise:

Estimate maximum batch size, that fits Colab GPU memory without and with checkpointing until it raise OOM (Out of memory) error.

| Without checkpointing | With checkpointing |
|:---:|:---:|
| ? | ? |

